# **Models from scratch**

[<font color='steelblue'>1. - __EDA__</font>](#one-bullet) <br>
    [<font color='steelblue'>1.1. - Import Libraries</font>](#two-bullet) <br>
    [<font color='steelblue'>1.2. - Import Data Files</font>](#three-bullet) <br>
    [<font color='steelblue'>1.3. - PreProcessing</font>](#four-bullet) <br>
    [<font color='steelblue'>1.4. - Removing Outliers</font>](#five-bullet) <br>
    [<font color='steelblue'>1.5. - Augmentation</font>](#six-bullet) <br>

[<font color='steelblue'>2. - __Models__</font>](#seven-bullet) <br>
    [<font color='steelblue'>2.1. - PreModel</font>](#eight-bullet) <br>
    [<font color='steelblue'>2.2. - Sequential</font>](#nine-bullet) <br>
    [<font color='steelblue'>2.3. - Functional</font>](#ten-bullet) <br>
    [<font color='steelblue'>2.4. - Functional with more layers</font>](#eleven-bullet) <br>
    [<font color='steelblue'>2.5. - Class imbalance - last try</font>](#twelve-bullet) <br>

Group 07
|Name | Number |
|----|----|
|Fábio dos Santos| 2024|
|Joana Rodrigues| 20240603|
|Mara Simões| 20240326|
|Matilde Street| 20240523|
|Rafael Borges| 20240497|

<hr>
<a class="anchor" id="one-bullet"> 
<d style="color:white;">

## 1. EDA
</a> 
</d>   

<a class="anchor" id="two-bullet"> 
<d style="color:white;">

### 1.1. Import Libraries
</a> 
</d>   

In [ ]:
import math
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
import pickle

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import add
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, AUC, SparseCategoricalAccuracy
from keras.layers import LeakyReLU
from keras.utils import to_categorical
from keras.metrics import F1Score
from keras.optimizers import Adam, RMSprop
import keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping

# Preprocessing 
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

# Outlier Detection
import hdbscan
from tensorflow.keras.applications import VGG16
from transformers import CLIPProcessor, TFCLIPModel
from sklearn.ensemble import IsolationForest
from sklearn.metrics.pairwise import cosine_distances
import os
import shutil

# Augmentation
from keras.layers import RandomBrightness, RandomContrast, RandomFlip, RandomRotation, RandomSaturation, Pipeline

# Bayesian optimization
#from keras_tuner import BayesianOptimization
#from keras.callbacks import Callback, EarlyStopping
#import wandb
#from wandb.integration.keras import WandbCallback

<a class="anchor" id="three-bullet"> 
<d style="color:white;">

### 1.2. Import Data Files
</a> 
</d>  
Firstly we create the path and after we will load the data files

In [3]:
path = Path('.') / 'data' / 'metadata.csv'

df = pd.read_csv(path)

# Display the first few rows of the dataframe to verify
df

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...
...,...,...,...,...,...,...,...
11978,1fa96ea5-32fa-4a25-b8d2-fa99f6e2cb89,29734618,1011315,animalia,chordata,leporidae,chordata_leporidae/29734618_1011315_eol-full-s...
11979,628bf2b4-6ecc-4017-a8e6-4306849e0cfc,29972861,1056842,animalia,chordata,emydidae,chordata_emydidae/29972861_1056842_eol-full-si...
11980,0ecfdec9-b1cd-4d43-96fc-2f8889ec1ad9,30134195,52572074,animalia,chordata,dasyatidae,chordata_dasyatidae/30134195_52572074_eol-full...
11981,27fdb1e9-c5fb-459a-8b6a-6fb222b1c512,9474963,46559139,animalia,chordata,mustelidae,chordata_mustelidae/9474963_46559139_eol-full-...


In the previous notebook we got the data splited.
In this we import it and for the train we import the 3 different strategies we created in the previous notebook. The three of them were tested with the models and the best is kept uncommented.
- original train with no modifications
- train with removal of 'outliers' based on the clip model (best results from the three different approaches tried in the previous notebook)
- train with removal of outliers and augmentation on the minority classes.

In [4]:
def get_image_paths_and_labels(base_path):
    base = Path(base_path)
    image_paths = list(base.rglob("*.jpg"))  
    data = {
        "filepath": [str(p) for p in image_paths],
        "label": [p.parent.name for p in image_paths]
    }
    return pd.DataFrame(data)


# Get the paths and labels for train, val, and test sets
dest_root = Path('.') / "data_split"
#train_df = get_image_paths_and_labels(dest_root / 'train')
train_df = get_image_paths_and_labels(Path('.')/'data_clean_clip')
train_df_aug = get_image_paths_and_labels(Path('.') / 'data_aug_clip')
val_df = get_image_paths_and_labels(dest_root / 'val')
test_df = get_image_paths_and_labels(dest_root / 'test')


<a class="anchor" id="four-bullet"> 
<d style="color:white;">

### 1.3. PreProcessing
</a> 
</d>
After the load of the data we proceed to do the remaining preprocessing steps here. 

#### 1.3.1. Encoder
Encoder: We load a pickle that saves the encoder created in the previous notebook for the train to avoid data leakage. We apply this to the validation and the test dataset so they all have the same values for the same classes.
After this we save the paths and the labels for each of the subset of the data to later use.

In [5]:
encoder = pickle.load(open("encoder.pkl", 'rb'))
val_df["label_encoded"] = encoder.transform(val_df["label"])
test_df["label_encoded"] = encoder.transform(test_df["label"])

In [6]:

train_labels = tf.constant(train_df["label"].astype(int).values)
train_paths = tf.constant(train_df["filepath"].values)

train_labels_aug = tf.constant(train_df_aug["label"].astype(int).values)
train_paths_aug = tf.constant(train_df_aug["filepath"].astype(str).values)

val_paths = tf.constant(val_df["filepath"].values)
val_labels = tf.constant(val_df["label_encoded"].values)

test_paths = tf.constant(test_df["filepath"].values)
test_labels = tf.constant(test_df["label_encoded"].values)


train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset_aug = tf.data.Dataset.from_tensor_slices((train_paths_aug, train_labels_aug))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

#### 1.3.2. Padding function     
This adds extra pixels to the pictures that are not a specific sizes (0s or a constant value) to avoid distortion of images and loss of information.

In [7]:
def resize_with_padding(image, target_height=224, target_width=224):
    shape = tf.shape(image)[:2] 
    height, width = tf.cast(shape[0], tf.float32), tf.cast(shape[1], tf.float32)

    scale = tf.minimum(target_width / width, target_height / height)
    new_height = tf.cast(height * scale, tf.int32)
    new_width = tf.cast(width * scale, tf.int32)

    resized_image = tf.image.resize(image, [new_height, new_width])

    pad_height = (target_height - new_height) // 2
    pad_width = (target_width - new_width) // 2
    padded_image = tf.image.pad_to_bounding_box(resized_image, pad_height, pad_width, target_height, target_width)

    return padded_image

#### 1.3.3. Preprocessing application
In here we create a function `decode_img` that will apply several different preprocessing steps to the images:
1. Reads the images
2. Decodes them from the JPEG format to be used as a tensor
3. Zooms (crops) the image 80% so that if the animal is too far away it can be read more clearly.
4. Usage of the padding function created previously.
5. Normalization to help the model train better (consistent scale, no bias)
6. Labels turned into one-hot encoding because it works best in NN with multiclass classification.


In [8]:
# Normalizing with 2 different methods to see which one works better
normalization_layer = tf.keras.layers.Rescaling(1./255) # 0-1 normalization
# normalization_layer = tf.keras.layers.Rescaling(1./127.5, offset=-1) # -1 to 1 normalization

def decode_img(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.central_crop(img, central_fraction=0.8)
    img = resize_with_padding(img) 
    #img = tf.image.resize(img, [224, 224]) # this was previously applied but the results were not as good
    img = normalization_layer(img)
    label = tf.one_hot(label, depth=202)
    return img, label


train_dataset = train_dataset.map(decode_img).batch(32).prefetch(tf.data.AUTOTUNE)
train_dataset_aug = train_dataset_aug.map(decode_img).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(decode_img).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(decode_img).batch(32).prefetch(tf.data.AUTOTUNE)



<a class="anchor" id="five-bullet"> 
<d style="color:white;">

### 1.4. Removing Outliers
</a> 
</d> 


- 1st with DBSCAN
- 2nd with VGG16
- 3rd with CLIP

We did it in the "eda" notebook and the best method was with CLIP, so from now on we will use the train dataset without the outliers identified with CLIP

In [9]:
# dest_root = Path('.') 
# # data_clean_dbscan = get_image_paths_and_labels(dest_root / 'data_clean_dbscan')
# # data_clean_vgg16 = get_image_paths_and_labels(dest_root / 'data_clean_vgg16')
# data_clean_clip = get_image_paths_and_labels(dest_root / 'data_clean_clip')
# train_dataset = data_clean_clip
# train_df["label_encoded"] = encoder.transform(train_df["label"])
# train_paths = tf.constant(train_df["filepath"].values)
# train_labels = tf.constant(train_df["label_encoded"].values)
# train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))


<a class="anchor" id="six-bullet"> 
<d style="color:white;">

### 1.4. Augmentation
</a> 
</d>   

Augmentation is important because it gives the model more variety in the images it sees. We used it before to help with the minority classes, but this time we applied it to all classes to try both approaches.

In [10]:
augmentation_layer = Pipeline([
    RandomBrightness(factor=0.1, value_range=(0.0, 1.0)),
    RandomSaturation(factor=0.2),
    RandomContrast(factor=0.2),
    RandomFlip(),
    RandomRotation(0.1)
])

In [11]:
def show_random_original_and_augmented(dataset, augmentation_layer):
    dataset = dataset.shuffle(1000)  # embaralha o dataset

    for images, _ in dataset.take(1):
        image = images[0]  # pega a primeira imagem do batch embaralhado
        augmented_image = augmentation_layer(tf.expand_dims(image, 0))[0]

        plt.figure(figsize=(10, 5))

        # Original
        plt.subplot(1, 2, 1)
        plt.imshow(image.numpy())
        plt.title("Original Image")
        plt.axis("off")

        # Augmented
        plt.subplot(1, 2, 2)
        plt.imshow(augmented_image.numpy())
        plt.title("Augmented Image")
        plt.axis("off")

        plt.show()
        break

In [12]:
#show_random_original_and_augmented(train_dataset, augmentation_layer)

<a class="anchor" id="six-bullet"> 
<d style="color:white;">

### 1.5. Class weights
</a> 
</d>   
Another technique to fight class imbalance tried is the class weights.
This, when applied to the model, makes the images from the minority classes have more weight in the decision.

In [13]:
train_labels_np = np.array(train_labels)

# Compute the class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_np),
    y=train_labels_np
)

# Create a dictionary mapping class indices to their weights
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
class_weight_dict


{0: 0.3878825382538254,
 1: 1.8963146314631463,
 2: 0.9752475247524752,
 3: 1.8963146314631463,
 4: 1.8963146314631463,
 5: 0.22021718300862345,
 6: 0.9481573157315731,
 7: 1.8963146314631463,
 8: 1.8963146314631463,
 9: 1.8963146314631463,
 10: 1.8963146314631463,
 11: 1.8963146314631463,
 12: 1.8963146314631463,
 13: 1.8963146314631463,
 14: 1.8963146314631463,
 15: 1.8963146314631463,
 16: 1.8963146314631463,
 17: 0.4876237623762376,
 18: 0.7111179867986799,
 19: 0.9481573157315731,
 20: 2.0078625509609784,
 21: 2.1333539603960396,
 22: 1.8963146314631463,
 23: 0.9481573157315731,
 24: 0.9481573157315731,
 25: 0.3923409582337544,
 26: 0.9481573157315731,
 27: 0.9481573157315731,
 28: 0.9481573157315731,
 29: 2.1333539603960396,
 30: 0.4015725101921957,
 31: 1.8963146314631463,
 32: 1.8963146314631463,
 33: 1.8963146314631463,
 34: 0.9752475247524752,
 35: 1.8963146314631463,
 36: 0.2687690028845404,
 37: 1.8963146314631463,
 38: 0.2994180997047073,
 39: 0.37926292629262925,
 40: 0.9

<hr>
<a class="anchor" id="seven-bullet"> 
<d style="color:white;">

## 2. Models
</a> 
</d>   

<a class="anchor" id="eight-bullet"> 
<d style="color:white;">

### 2.1. PreModel
</a> 
</d>   
Definition of metrics and input shape. Reasons for choice of metrics:

|**Metric used** | **Reason**|
|----------------|----------------------------------------------------------------------------------------------|
|**Accuracy**| Gives an overall idea of performance, but not very good because it's misleading with such an imbalanced dataset.|
|**Precision**| Says how often a model is correct when it says it's a specific class.|
|**Recall**| Tell how many times an image classification is right in comparison to the total there is of that image |
|**F1 Score**| Balances precision and recall. good for our imbalanced 202-class problem.|
|**Loss** (later shown)| Good to see how well the model is learning. The lower the best (model more confident and performative)|



In [14]:
input_shape = (224, 224, 3)
n_classes = 202


categorical_accuracy = CategoricalAccuracy(name="accuracy")
precision = CategoricalAccuracy(name="precision")
recall = CategoricalAccuracy(name="recall")
f1_score = F1Score(average="macro", name="f1_score")
metrics = [categorical_accuracy,precision, recall, f1_score]

#### Functions to Investigate the Class Prediction of the Dataset

1. **Class Report**: This function generates a detailed report on the model’s predictions, showing the correct and incorrect predictions, as well as the certainty levels associated with each class.
   
2. **Most predictions**: Most frequent predicted classes in the dataset --> may be confunding the dataset
 
3. **no_predictions_or_incorrect**: Classes that are not being predicted at all or are incorrectly predicted --> useful for maybe unbalancing metrics...
   
4. **Average Certainty Calculation**:  average certainty (as a percentage) for correct and incorrect predictions (how confident the model is in the predictions) --> useful maybe for thresholding (maybe for the folder others???)


In [15]:
# def class_report(model):

#     # Getting the true labels from the dataset
#     y_true = []
#     for images, labels in test_dataset:
#         y_true.extend(labels.numpy())  # Store true labels

#     # Get the prediction probabilities from the model
#     y_pred_prob = model.predict(test_dataset, verbose=0)

#     # Get the predicted class labels (class with the highest probability)
#     y_pred_classes = np.argmax(y_pred_prob, axis=1)

#     # Convert y_true to numpy and get the class labels if one-hot encoded
#     y_true = np.argmax(np.array(y_true), axis=1)

#     # Initialize counters for correct and incorrect predictions and their respective certainties
#     correct_predictions = [0] * len(np.unique(y_true))
#     incorrect_predictions = [0] * len(np.unique(y_true))
#     correct_certainties = [0] * len(np.unique(y_true))
#     incorrect_certainties = [0] * len(np.unique(y_true))

#     # Calculate predictions and certainties for each class
#     for class_idx in range(len(np.unique(y_true))):  # Loop over all classes
#         correct_mask = (y_true == class_idx) & (y_pred_classes == class_idx)
#         incorrect_mask = (y_true == class_idx) & (y_pred_classes != class_idx)

#         if np.any(correct_mask):
#             correct_certainty = np.max(y_pred_prob[correct_mask], axis=1)
#             correct_certainties[class_idx] = np.mean(correct_certainty)  # Mean certainty for the class
#             correct_predictions[class_idx] = np.sum(correct_mask)

#         if np.any(incorrect_mask):
#             incorrect_certainty = np.max(y_pred_prob[incorrect_mask], axis=1)
#             incorrect_certainties[class_idx] = np.mean(incorrect_certainty)  # Mean certainty for the class
#             incorrect_predictions[class_idx] = np.sum(incorrect_mask)

#     # Create a DataFrame with the information
#     class_report = pd.DataFrame({
#         'True Class': y_true,
#         'Predicted Class': y_pred_classes,
#         'Correct Predictions': correct_predictions,
#         'Incorrect Predictions': incorrect_predictions,
#         'Certainty of Correct': correct_certainties,
#         'Certainty of Incorrect': incorrect_certainties
#     })

#     # Set 'True Class' as the index
#     class_report.set_index('True Class', inplace=True)

#     # Sort the DataFrame by 'Certainty of Correct' in descending order
#     class_report['Sort Order'] = class_report['Certainty of Correct'].replace(0, -1)
#     class_report = class_report.sort_values(by=['Sort Order', 'Certainty of Incorrect'], ascending=[False, False])
#     class_report.drop('Sort Order', axis=1, inplace=True)

#     return class_report


# def most_predictions(y_pred_classes):
#     # Contar a frequência de cada classe predita
#     class_counts = np.bincount(y_pred_classes)

#     # Obter os índices das 5 classes mais frequentes
#     top_5_classes = np.argsort(class_counts)[-5:][::-1]

#     # Obter a contagem de vezes que cada uma das 5 classes foi predita
#     top_5_class_counts = class_counts[top_5_classes]

#     print("Top 5 classes more predicted:")
#     for class_id, count in zip(top_5_classes, top_5_class_counts):
#         print(f"Class {class_id}: {count} times")
#     return

# def no_predictions_or_incorrect(y_true, y_pred_classes):
#     # Count the frequency of each predicted class
#     class_counts = np.bincount(y_pred_classes)

#     # Find classes that have zero predictions
#     zero_pred_classes = np.where(class_counts == 0)[0]

#     # Count how many classes have no predictions
#     num_zero_pred_classes = len(zero_pred_classes)

#     # Check how many classes are not being predicted correctly (excluding zero-predicted classes)
#     incorrect_predictions_mask = y_true != y_pred_classes
#     incorrect_classes = np.unique(y_true[incorrect_predictions_mask])

#     # Filter out incorrect classes that are in zero_pred_classes
#     incorrect_classes = [cls for cls in incorrect_classes if cls not in zero_pred_classes]

#     # Print the results in the required format
#     print(f"{num_zero_pred_classes} classes are not being predicted at all: {zero_pred_classes}")
#     print(f"{len(incorrect_classes) } classes are not being predicted correctly: {incorrect_classes}")

#     return


# def certainties_averages(correct_certainties, incorrect_certainties):

#     # Filter out zeros from the correct and incorrect certainties
#     filtered_correct_certainties = [certainty for certainty in correct_certainties if certainty > 0]
#     filtered_incorrect_certainties = [certainty for certainty in incorrect_certainties if certainty > 0]

#     # Calculate the average percentage of certainty for correct and incorrect predictions, excluding zeros
#     average_correct_certainty = np.mean(filtered_correct_certainties) * 100  # Multiply by 100 to get percentage
#     average_incorrect_certainty = np.mean(filtered_incorrect_certainties) * 100  # Multiply by 100 to get percentage

#     # Print the results
#     print(f"Average Correct Certainty (excluding zeros): {average_correct_certainty:.2f}%")
#     print(f"Average Incorrect Certainty (excluding zeros): {average_incorrect_certainty:.2f}%")
    
#     return


<a class="anchor" id="nine-bullet"> 
<d style="color:white;">

### 2.2. Sequential
</a> 
</d>   

**Baseline model**

In [ ]:
model = Sequential(
    layers=[
        Input(shape=input_shape),
        # augmentation_layer, 
        Conv2D(filters=24, kernel_size=(3, 3), activation="relu"),  
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=48, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(n_classes, activation="softmax"),
    ],
    name="sequential")

In [15]:
# Compile the model 
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=metrics
)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    class_weight=class_weight_dict
)

Epoch 1/10


1042/1042 ━━━━━━━━━━━━━━━━━━━━ 883s 838ms/step - accuracy: 0.0302 - f1_score: 0.0023 - loss: 5.5356 - precision: 0.0302 - recall: 0.0302 - val_accuracy: 0.0050 - val_f1_score: 4.9381e-05 - val_loss: 5.3571 - val_precision: 0.0050 - val_recall: 0.0050
Epoch 2/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 801s 758ms/step - accuracy: 0.0363 - f1_score: 0.0102 - loss: 4.7523 - precision: 0.0363 - recall: 0.0363 - val_accuracy: 0.0046 - val_f1_score: 1.5978e-04 - val_loss: 5.3349 - val_precision: 0.0046 - val_recall: 0.0046
Epoch 3/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 888s 849ms/step - accuracy: 0.0307 - f1_score: 0.0077 - loss: 4.9913 - precision: 0.0307 - recall: 0.0307 - val_accuracy: 0.0079 - val_f1_score: 0.0022 - val_loss: 5.3513 - val_precision: 0.0079 - val_recall: 0.0079
Epoch 4/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 1025s 975ms/step - accuracy: 0.0226 - f1_score: 0.0063 - loss: 4.9799 - precision: 0.0226 - recall: 0.0226 - val_accuracy: 0.0071 - val_f1_score: 0.0011 - val_loss: 5.3856 - val_precision

In [ ]:
results = model.evaluate(
    test_dataset,
    return_dict=True,
    verbose=0 
)

print("Test set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")

Test set results:
accuracy: 0.0233
f1_score: 0.0155
loss: 7.9670
precision: 0.0233
recall: 0.0233


**Bayesian search**

In [ ]:
# # Global variable to track best F1 across all trials
# best_f1 = 0.0

# # Sweep configuration for W&B
# sweep_config = {
#     'method': 'bayes',  # Bayesian optimization
#     'name': 'SequentialModelSweep',
#     'metric': {
#         'name': 'val_f1_score',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'filters_1': {'values': [16, 32, 48, 64]},
#         'filters_2': {'values': [32, 64, 96, 128]},
#         'kernel_size': {'values': ['3x3', '5x5']},
#         'learning_rate': {'min': 1e-4, 'max': 1e-2},
#         'use_augmentation': {'values': [True, False]},
#         'activation': {'values': ['relu', 'leaky_relu']},
#         'optimizer': {'values': ['adam', 'rmsprop']},
#     }
# }

# # Initialize sweep
# sweep_id = wandb.sweep(sweep_config, project="dl_project")

# # Training function
# def train_model(config=None):
#     with wandb.init(config=config):
#         config = wandb.config

#         model = Sequential()
#         model.add(Input(shape=input_shape))

#         if config.use_augmentation:
#             model.add(augmentation_layer)

#         kernel_size = tuple(map(int, config.kernel_size.split("x")))
#         model.add(Conv2D(config.filters_1, kernel_size=kernel_size))
#         model.add(LeakyReLU() if config.activation == "leaky_relu" else tf.keras.layers.Activation(config.activation))
#         model.add(MaxPooling2D(pool_size=(2, 2)))

#         model.add(Conv2D(config.filters_2, kernel_size=kernel_size))
#         model.add(LeakyReLU() if config.activation == "leaky_relu" else tf.keras.layers.Activation(config.activation))
#         model.add(MaxPooling2D(pool_size=(2, 2)))

#         model.add(Flatten())
#         model.add(Dense(n_classes, activation="softmax"))

#         optimizer = Adam(learning_rate=config.learning_rate) if config.optimizer == "adam" else RMSprop(learning_rate=config.learning_rate)

#         model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=metrics)

#         early_stopping = EarlyStopping(monitor="val_f1_score", patience=3, restore_best_weights=True, mode="max")
#         wandb_callback = WandbCallback(save_graph=False, save_model=False)

#         history = model.fit(
#             train_dataset,
#             validation_data=val_dataset,
#             epochs=5,
#             class_weight=class_weight_dict,
#             callbacks=[early_stopping, wandb_callback]
#         )

#         # Save best model if this trial achieved a better f1
#         global best_f1
#         val_f1_scores = history.history.get("val_f1_score")
#         if val_f1_scores:
#             trial_best = max(val_f1_scores)
#             if trial_best > best_f1:
#                 best_f1 = trial_best
#                 model.save("best_model.keras")
#                 print(f"\n Saved new best model with val_f1_score = {best_f1:.4f}")

# # Run the sweep
# wandb.agent(sweep_id, function=train_model, count=50)

In [ ]:
# # check best model parameters
# api = wandb.Api()
# sweep = api.sweep("matildestreet-nova-ims/dl_project/4xk38eju")
# runs = sweep.runs
# best_run = max(runs, key=lambda r: r.summary.get("val_f1_score", 0))

# print("Best run config:")
# for key, value in best_run.config.items():
#     print(f"{key}: {value}")



# # output - best was trial 49
# Best run config:
# filters_1: 32
# filters_2: 32
# optimizer: rmsprop
# activation: leaky_relu
# kernel_size: 5x5
# learning_rate: 0.001526590318714996
# use_augmentation: False

In [16]:
best_model = Sequential(
    layers=[
        Input(shape=input_shape),
        Conv2D(filters=32, kernel_size=(5, 5), activation="leaky_relu"),  
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(5, 5), activation="leaky_relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(n_classes, activation="softmax"),
    ],
    name="sequential"
)

In [17]:
# Compile with best optimizer and learning rate
optimizer = RMSprop(learning_rate=0.001526590318714996)

# Compile the model 
best_model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=metrics
)

In [ ]:


early_stop = EarlyStopping(
    monitor='val_f1_score',   # nome da métrica que defines (ver abaixo)
    mode='max',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [19]:
# Train the model
history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs= 50,
    class_weight=class_weight_dict,
    callbacks=[early_stop])

Epoch 1/50


 56/216 ━━━━━━━━━━━━━━━━━━━━ 2:50 1s/step - accuracy: 0.2193 - f1_score: 0.0126 - loss: 12.3761 - precision: 0.2193 - recall: 0.2193

KeyboardInterrupt: 

In [19]:
results = best_model.evaluate(
    test_dataset,
    return_dict=True,
    verbose=0 
)

print("Test set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")

Test set results:
accuracy: 0.0337
f1_score: 0.0261
loss: 18.8094
precision: 0.0337
recall: 0.0337


the f1 score is higher after the bayesian optimization, however the difference is not that big and the loss is much higher

Trying with the dataaugmentation to see if the score gets better

In [ ]:
# Train the model
history = best_model.fit(
    train_dataset_aug,
    validation_data=val_dataset,
    epochs=10,
    class_weight=class_weight_dict
)

NameError: name 'best_model' is not defined

In [ ]:
results = best_model.evaluate(
    test_dataset,
    return_dict=True,
    verbose=0 
)

print("Test set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")

Test set results:
accuracy: 0.0079
f1_score: 0.0101
loss: 21.7057
precision: 0.0079
recall: 0.0079


**Baseline model**

In [ ]:
def functional_model():
    conv_layer_1 = layers.Conv2D(
        filters=3 * 8,
        kernel_size=(3, 3),
        activation="relu",
        name="conv_layer_1"
    )
    max_pool_layer_1 = layers.MaxPooling2D(pool_size=(2, 2), name="max_pool_layer_1")

    conv_layer_2 = layers.Conv2D(
        filters=3 * 16,
        kernel_size=(3, 3),
        name="conv_layer_2"
    )
    act_layer_2 = layers.LeakyReLU(negative_slope=0.3, name="act_layer_2")
    max_pool_layer_2 = layers.MaxPooling2D(pool_size=(2, 2), name="max_pool_layer_2")

    flatten_layer = layers.Flatten(name="flatten_layer")
    dense_layer = layers.Dense(
        n_classes,
        activation="softmax",
        name="classification_head"
    )

    inputs = layers.Input(shape=input_shape)
    x = inputs

    x = conv_layer_1(x)
    x = max_pool_layer_1(x)

    x = conv_layer_2(x)
    x = act_layer_2(x)
    x = max_pool_layer_2(x)

    x = flatten_layer(x)
    x = dense_layer(x)

    outputs = x

    return models.Model(inputs=inputs, outputs=outputs, name="my_tiny_functional_cnn")

In [ ]:
# Compile the model
model=functional_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10)

Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 47s 214ms/step - accuracy: 0.0300 - f1_score: 0.0140 - loss: 6.3319 - precision: 0.0300 - recall: 0.0300 - val_accuracy: 0.0100 - val_f1_score: 9.8314e-05 - val_loss: 7.5576 - val_precision: 0.0100 - val_recall: 0.0100
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 45s 210ms/step - accuracy: 0.0312 - f1_score: 0.0037 - loss: 5.6284 - precision: 0.0312 - recall: 0.0312 - val_accuracy: 0.0075 - val_f1_score: 7.3919e-05 - val_loss: 11.3704 - val_precision: 0.0075 - val_recall: 0.0075
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 45s 209ms/step - accuracy: 0.0291 - f1_score: 0.0053 - loss: 5.3975 - precision: 0.0291 - recall: 0.0291 - val_accuracy: 0.0092 - val_f1_score: 0.0018 - val_loss: 8.5347 - val_precision: 0.0092 - val_recall: 0.0092
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 45s 209ms/step - accuracy: 0.0436 - f1_score: 0.0144 - loss: 4.9761 - precision: 0.0436 - recall: 0.0436 - val_accuracy: 0.0171 - val_f1_score: 0.0060 - val_loss: 9.9364 - val_precision:

In [ ]:
results = model.evaluate(
    test_dataset,
    return_dict=True,
    verbose=0
)

print("Test set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")

Test set results:
accuracy: 0.0283
f1_score: 0.0232
loss: 13.1315
precision: 0.0283
recall: 0.0283


**Bayesian search**

In [68]:
# # Sweep configuration
# sweep_config = {
#     'method': 'bayes',
#     'name': 'FunctionalModelSweep',
#     'metric': {
#         'name': 'val_f1_score',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'filters_1': {'values': [16, 32, 48, 64]},
#         'filters_2': {'values': [32, 64, 96, 128]},
#         'kernel_size': {'values': ['3x3', '5x5']},
#         'learning_rate': {'min': 1e-4, 'max': 1e-2},
#         'use_augmentation': {'values': [True, False]},
#         'activation': {'values': ['relu', 'leaky_relu']},
#         'alpha': {'min': 0.1, 'max': 0.5},  # for LeakyReLU
#         'optimizer': {'values': ['adam', 'rmsprop']},
#     }
# }

# sweep_id = wandb.sweep(sweep_config, project="dl_project")

# # Training function
# def train_model(config=None):
#     with wandb.init(config=config):
#         config = wandb.config

#         # Layers
#         conv1 = layers.Conv2D(filters=config.filters_1, kernel_size=tuple(map(int, config.kernel_size.split('x'))), name='conv_1')
#         conv2 = layers.Conv2D(filters=config.filters_2, kernel_size=tuple(map(int, config.kernel_size.split('x'))), name='conv_2')
#         maxpool = layers.MaxPooling2D(pool_size=(2, 2))

#         activation_1 = layers.Activation('relu') if config.activation == "relu" else layers.LeakyReLU(alpha=config.alpha)
#         activation_2 = layers.Activation('relu') if config.activation == "relu" else layers.LeakyReLU(alpha=config.alpha)

#         flatten = layers.Flatten()
#         dense = layers.Dense(n_classes, activation='softmax', name='output')

#         inputs = layers.Input(shape=input_shape)
#         x = inputs

#         if config.use_augmentation:
#             x = augmentation_layer(x)

#         x = conv1(x)
#         x = maxpool(x)

#         x = conv2(x)
#         x = activation_2(x)
#         x = maxpool(x)

#         x = flatten(x)
#         outputs = dense(x)

#         model = models.Model(inputs=inputs, outputs=outputs)

#         optimizer = Adam(learning_rate=config.learning_rate) if config.optimizer == "adam" else RMSprop(learning_rate=config.learning_rate)
#         model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=metrics)

#         early_stopping = EarlyStopping(monitor='val_f1_score', patience=3, restore_best_weights=True, mode='max')

#         model.fit(
#             train_dataset,
#             validation_data=val_dataset,
#             epochs=10,
#             class_weight=class_weight_dict,
#             callbacks=[early_stopping, WandbCallback(save_model=False, save_graph=False)],
#             verbose=1
#         )

# # Start sweep
# wandb.agent(sweep_id, function=train_model, count=50)

In [67]:
# # Load W&B API and fetch sweep by project path and sweep ID
# api = wandb.Api()
# sweep = api.sweep("matildestreet-nova-ims/dl_project/7jkmks1l")  
# runs = sweep.runs

# # Get the run with the highest validation F1 score
# best_run = max(runs, key=lambda r: r.summary.get("val_f1_score", 0))

# # Print the configuration of the best run
# print("Best run config:")
# for key, value in best_run.config.items():
#     print(f"{key}: {value}")



# # output
# Best run config:
# alpha: 0.48096970546631235
# filters_1: 64
# filters_2: 32
# optimizer: rmsprop
# activation: leaky_relu
# kernel_size: 3x3
# learning_rate: 0.0005838087914898498
# use_augmentation: False

In [62]:
# Define the functional model with best parameters
def best_functional_model():
    inputs = layers.Input(shape=input_shape, name="input")

    # Conv layer 1
    x = layers.Conv2D(64, kernel_size=(3, 3), activation=None, name="conv_1")(inputs)
    x = layers.LeakyReLU(alpha=0.48, name="leaky_relu_1")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name="max_pool_1")(x)

    # Conv layer 2
    x = layers.Conv2D(32, kernel_size=(3, 3), activation=None, name="conv_2")(x)
    x = layers.LeakyReLU(alpha=0.48, name="leaky_relu_2")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name="max_pool_2")(x)

    # Flatten + classification head
    x = layers.Flatten(name="flatten")(x)
    outputs = layers.Dense(n_classes, activation="softmax", name="output")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="best_functional_model")
    return model

# Instantiate and compile the model
best_model = best_functional_model()
optimizer = optimizers.RMSprop(learning_rate=0.0005838087914898498)

best_model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=metrics  
)

# Train the model (sem class_weight)
history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

c:\USERS\PAULO\ANACONDA3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 3:45 1s/step - accuracy: 0.0312 - f1_score: 0.0135 - loss: 5.5261 - precision: 0.0312 - recall: 0.031 ━━━━━━━━━━━━━━━━━━━━ 1:03 299ms/step - accuracy: 0.0375 - f1_score: 0.0138 - loss: 4.1446 - precision: 0.0375 - recall: 0.03 ━━━━━━━━━━━━━━━━━━━━ 1:04 304ms/step - accuracy: 0.0426 - f1_score: 0.0140 - loss: 3.8233 - precision: 0.0426 - recall: 0.04 ━━━━━━━━━━━━━━━━━━━━ 1:03 303ms/step - accuracy: 0.0460 - f1_score: 0.0144 - loss: 3.8495 - precision: 0.0460 - recall: 0.04 ━━━━━━━━━━━━━━━━━━━━ 1:04 306ms/step - accuracy: 0.0503 - f1_score: 0.0152 - loss: 3.7741 - precision: 0.0503 - recall: 0.05 ━━━━━━━━━━━━━━━━━━━━ 1:02 300ms/step - accuracy: 0.0531 - f1_score: 0.0156 - loss: 3.9943 - precision: 0.0531 - recall: 0.05 ━━━━━━━━━━━━━━━━━━━━ 1:02 300ms/step - accuracy: 0.0550 - f1_score: 0.0159 - loss: 4.1693 - precision: 0.0550 - recall: 0.05 ━━━━━━━━━━━━━━━━━━━━ 1:02 304ms/step - accuracy: 0.0565 - f1_score: 0.0162 - loss: 4.2824 - precision: 0.056

In [63]:
# Evaluate
results = best_model.evaluate(test_dataset, return_dict=True, verbose=0)
print("\nTest set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")


Test set results:
accuracy: 0.0325
f1_score: 0.0341
loss: 7.7036
precision: 0.0325
recall: 0.0325


In [64]:
# same model with class_weight

# Instantiate and compile the model
best_model = best_functional_model()
optimizer = optimizers.RMSprop(learning_rate=0.0005838087914898498)

best_model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=metrics  
)

# Train the model (sem class_weight)
history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    class_weight=class_weight_dict
)

Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 3:53 1s/step - accuracy: 0.0321 - f1_score: 0.0325 - loss: 2.0511 - precision: 0.0321 - recall: 0.032 ━━━━━━━━━━━━━━━━━━━━ 1:49 512ms/step - accuracy: 0.0383 - f1_score: 0.0330 - loss: 1.5383 - precision: 0.0383 - recall: 0.03 ━━━━━━━━━━━━━━━━━━━━ 1:45 498ms/step - accuracy: 0.0434 - f1_score: 0.0333 - loss: 1.8697 - precision: 0.0434 - recall: 0.04 ━━━━━━━━━━━━━━━━━━━━ 1:33 441ms/step - accuracy: 0.0468 - f1_score: 0.0338 - loss: 2.4309 - precision: 0.0468 - recall: 0.04 ━━━━━━━━━━━━━━━━━━━━ 1:22 391ms/step - accuracy: 0.0494 - f1_score: 0.0345 - loss: 2.7066 - precision: 0.0494 - recall: 0.04 ━━━━━━━━━━━━━━━━━━━━ 1:17 371ms/step - accuracy: 0.0510 - f1_score: 0.0348 - loss: 3.0991 - precision: 0.0510 - recall: 0.05 ━━━━━━━━━━━━━━━━━━━━ 1:14 356ms/step - accuracy: 0.0522 - f1_score: 0.0351 - loss: 3.4677 - precision: 0.0522 - recall: 0.05 ━━━━━━━━━━━━━━━━━━━━ 1:11 346ms/step - accuracy: 0.0529 - f1_score: 0.0353 - loss: 3.6877 - precision: 0.052

In [65]:
# Evaluate
results = best_model.evaluate(test_dataset, return_dict=True, verbose=0)
print("\nTest set results:")
for metric_name, value in results.items():
    print(f"{metric_name}: {value:.4f}")


Test set results:
accuracy: 0.0271
f1_score: 0.0294
loss: 9.6477
precision: 0.0271
recall: 0.0271


<a class="anchor" id="eleven-bullet"> 
<d style="color:white;">

### 2.4. Functional with more layers
</a> 
</d>  

<a class="anchor" id="twelve-bullet"> 
<d style="color:white;">

### 2.5. Class Imbalance - last try
</a> 
</d>  